In [0]:
%pip install databricks-feature_engineering

In [0]:
%restart_python

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import json
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
import joblib
from datetime import datetime, date
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup

In [0]:
dbutils.widgets.text("training_csv", "")

In [0]:
training_csv = dbutils.widgets.get("training_csv")

In [0]:
df = pd.read_csv(training_csv)
df.shape

In [0]:
df.head()

In [0]:
df.columns.to_list()

In [0]:

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')

# Convert dates
df['status_date'] = pd.to_datetime(df['lr_created_date'], errors='coerce')

# Create month-year column
df['month_year'] = df['status_date'].dt.strftime('%m-%Y')

In [0]:
# Assign type based on precise date ranges
# For X_train use type='train' (2023-06 to 2024-09)
# For X_test use type='test' (2024-10 full month) 
# For Live use type='live' (2024-11 full month)
# Rest will be deleted
# Assign type based on date ranges
def assign_type(date):
    if pd.Timestamp('2023-06-01') <= date < pd.Timestamp('2024-10-01'):
        return 'train'
    elif pd.Timestamp('2024-10-01') <= date < pd.Timestamp('2024-11-01'):
        return 'test'
    elif pd.Timestamp('2024-11-01') <= date < pd.Timestamp('2024-12-01'):
        return 'live'
    else:
        return 'delete'

df['type'] = df['status_date'].apply(assign_type)

# Show counts
print(df['type'].value_counts())

# Filter and save datasets
save_path = "/Volumes/ispl_databricks/default/training/MW_Train/"  # Databricks-compatible path

# for dataset_type in ['train', 'test', 'live']:
#     subset = df[df['type'] == dataset_type]
#     subset.to_csv(f"{save_path}{dataset_type}.csv", index=False)


In [0]:
# Create monthwise distribution visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Create a monthwise breakdown
df['year_month'] = df['status_date'].dt.to_period('M')

# Get monthwise distribution by type
monthly_dist = df.groupby(['year_month', 'type']).size().unstack(fill_value=0)

# Create the visualization
plt.figure(figsize=(15, 8))

# Plot stacked bar chart
monthly_dist.plot(kind='bar', stacked=True, figsize=(15, 8), 
                  color=['#1f77b4', '#ff7f0e', '#2ca02c'])  # Blue, Orange, Green

plt.title('Monthwise Distribution of Train, Test, and Live Data', fontsize=16, fontweight='bold')
plt.xlabel('Month-Year', fontsize=12)
plt.ylabel('Number of Records', fontsize=12)
plt.legend(title='Data Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Print detailed monthwise breakdown
print("Monthwise Distribution:")
print("=" * 50)
print(monthly_dist)
print("\n" + "=" * 50)

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total records: {len(df):,}")
print(f"Date range: {df['status_date'].min()} to {df['status_date'].max()}")
print(f"Train data: {len(df[df['type'] == 'train']):,} records")
print(f"Test data: {len(df[df['type'] == 'test']):,} records") 
print(f"Live data: {len(df[df['type'] == 'live']):,} records")


In [0]:
# Create a copy of the dataframe
final = df.copy()


# Convert '_is_' and '_exist' columns to category
is_columns = [col for col in final.columns if '_is_' in col]
exist_columns = [col for col in final.columns if col.endswith('_exist')]
cat_columns = list(set(is_columns + exist_columns))  # combine lists

# Convert all categorical columns at once
final[cat_columns] = final[cat_columns].astype('category')
print(f"Number of categorical columns converted: {len(cat_columns)}")


# Replace common boolean/string values
replace_dict = {
    "true": 1, "TRUE": 1, True: 1,
    "false": 0, "FALSE": 0, False: 0,
    "na": np.nan,
    "error": np.nan
}

final.replace(replace_dict, inplace=True)


# Optional: Custom mapping for specific columns (example)
# Example for a column like whatsapp_is_business
# final['s3m_phone_data_primary_data_whatsapp_is_business'] = final['s3m_phone_data_primary_data_whatsapp_is_business'].map(
#     lambda x: 1 if str(x) in ['1', '1.0'] else (0 if str(x) in ['0', '0.0'] else x)
# )


# Verify changes
print(final[cat_columns].head())
print(final[cat_columns].dtypes.value_counts())

In [0]:
def calculate_age(dob_str):
    """Calculate age from DD-MM-YYYY string. Returns -1 if invalid or <18."""
    if pd.isna(dob_str):
        return -1
    try:
        born = datetime.strptime(str(dob_str), "%d-%m-%Y").date()
        today = date.today()
        age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        return age if age >= 18 else -1
    except Exception:
        return -1

# Apply function vectorized
final['Age'] = final['bue_dob'].apply(calculate_age)

# Optional: check distribution
print(final['Age'].value_counts(dropna=False).head())

In [0]:
# Convert all object type columns to category
object_columns = final.select_dtypes(include='object').columns
final[object_columns] = final[object_columns].astype('category')

# Identify categorical columns (both category + object, just in case)
categorical_cols = final.select_dtypes(include=['category']).columns

# Drop columns with > 50 unique categories (except 'required_loan_id')
high_cardinality = [
    col for col in categorical_cols 
    if col != 'required_loan_id' and final[col].nunique(dropna=True) > 50
]

final.drop(columns=high_cardinality, inplace=True)

print(f"Converted {len(object_columns)} object columns to category")
print(f"Dropped {len(high_cardinality)} high-cardinality categorical columns")


In [0]:
ratio_features = {
    "bue_perc_no_of_open_loans": ("bue_no_of_open_loans", "bue_no_of_loans"),
    "bue_perc_no_of_open_loans_lst_6months": ("bue_no_of_open_loans_lst_6months", "bue_no_of_loans"),
    "bue_perc_no_of_cc_loans": ("bue_no_of_cc_loans", "bue_no_of_loans"),
    "bue_perc_no_of_cc_open_loans": ("bue_no_of_cc_open_loans", "bue_no_of_open_loans"),
    "bue_perc_no_of_auto_open_loans": ("bue_no_of_auto_open_loans", "bue_no_of_open_loans"),
    "bue_perc_no_of_consumer_loans": ("bue_no_of_consumer_loans", "bue_no_of_loans"),
    "bue_perc_no_of_consumer_open_loans": ("bue_no_of_consumer_open_loans", "bue_no_of_open_loans"),
    "bue_perc_no_of_personal_loans": ("bue_no_of_personal_loans", "bue_no_of_loans"),
    "bue_perc_no_of_gold_loans": ("bue_no_of_gold_loans", "bue_no_of_loans"),
}

# Create ratios safely (avoid division by zero)
for new_col, (num, denom) in ratio_features.items():
    final[new_col] = np.where(
        final[denom] > 0,
        np.round(final[num] / final[denom], 2),
        np.nan  # assign NaN if denominator is 0 or missing
    )

# Inspect a sample of created features along with target
print(final[list(ratio_features.keys()) + ['bue_min_count_of_emi', 'target_30_dpd']].head())

In [0]:
# Columns to drop
drop_columns = [
    'source','data_from','_merge','task_id','status','occupation',
    'lr_created_date','bue_dob','disbursement_completion_date',
    'month_year','Age','bue_min_count_of_emi'
]

# Inspect example column before dropping (optional)
#print(final['s3e_email_data_linked_data_skype_creation_time'].value_counts(dropna=False).head())

# Save dtypes for reference
dtype_df = final.dtypes.reset_index()
dtype_df.columns = ['column_name', 'dtype']
dtype_df.to_csv("/Volumes/ispl_databricks/default/training/MW_Train/data_types.csv", index=False)

# Drop unwanted columns
base_df1 = final.drop(columns=drop_columns, axis=1, errors='ignore')

# Separate numeric and categorical
df_num = base_df1.select_dtypes(include=['float64', 'int64', 'int32']).copy()
df_cat = base_df1.select_dtypes(include=['object', 'boolean', 'category']).copy()

# Fill missing numeric values with 0
df_num = df_num.fillna(0)

# Combine numeric + categorical back
base_df2 = pd.concat([df_num, df_cat], axis=1)

# Rename column safely
base_df2 = base_df2.rename(columns={'required_loan_id': 'loan_id'})

print("Final dataset shape:", base_df2.shape)

In [0]:
# === Column Type Analysis ===
print("=== Column Type Analysis ===")
print(f"Total columns in base_df2: {len(base_df2.columns)}")

# Identify categorical columns
categorical_columns = base_df2.select_dtypes(include=['object', 'category', 'boolean']).columns.tolist()
print(f"Number of categorical columns: {len(categorical_columns)}")
if categorical_columns:
    print("Sample categorical columns:", categorical_columns[:10], "..." if len(categorical_columns) > 10 else "")

# Identify numerical columns
numerical_columns = base_df2.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
print(f"Number of numerical columns: {len(numerical_columns)}")
if numerical_columns:
    print("Sample numerical columns:", numerical_columns[:10], "..." if len(numerical_columns) > 10 else "")

# Identify other/unsupported dtypes
other_columns = base_df2.select_dtypes(
    exclude=['object', 'category', 'boolean', 'int64', 'float64', 'int32', 'float32']
).columns.tolist()
print(f"Number of other type columns: {len(other_columns)}")
if other_columns:
    print("Other type columns:", other_columns)

# Summary by dtype
print("\n=== Data Type Summary ===")
print(base_df2.dtypes.value_counts())

In [0]:
# Calculate percentage of missing values per column
null_percentages = base_df2.isnull().mean() * 100

# Identify columns with >= 40% missing values
columns_to_drop = null_percentages[null_percentages >= 40].index.tolist()

# Drop those columns safely
base_df2.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Log info
print(f"Dropped {len(columns_to_drop)} columns with >=40% missing values:")
if columns_to_drop:
    print(columns_to_drop[:10], "..." if len(columns_to_drop) > 10 else "")

In [0]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in base_df2.columns:
    dtype = base_df2[col].dtype

    if col in ["loan_id","type"]:
        continue

    if dtype.name in ["category", "object"]:
        le = LabelEncoder()
        base_df2[col] = le.fit_transform(
            base_df2[col].astype(str)
        )
        label_encoders[col] = le

In [0]:
base_df2['type']

In [0]:
base_df_spark = spark.createDataFrame(base_df2)

In [0]:
base_df_feature_store = base_df_spark.drop('target_30_dpd','type','target_source')
base_4_feature_store = base_df_feature_store.drop_duplicates(['loan_id'])
base_4_spark = base_df_spark.drop_duplicates(['loan_id'])

In [0]:
fe = FeatureEngineeringClient()
fe.create_table(
  name="ispl_databricks.model_logs.mw_feature_store_500",
  primary_keys=["loan_id"],
  df=base_4_feature_store,
  description="Feature table for the bank"
)
fe.write_table(
    name="ispl_databricks.model_logs.mw_feature_store_500",
    df=base_4_feature_store,                 # Spark or pandas DataFrame
    mode="merge"           # works like upsert (recommended)
)

In [0]:
base_4_spark.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("ispl_databricks.model_logs.base_df_500features_updated")